In [ ]:
import numpy as np
import pandas as pd 

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))


In [ ]:
!pip install split_folders

In [ ]:
import split_folders

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

In [ ]:
os.makedirs('output')
os.makedirs('output/train')
os.makedirs('output/val')
os.makedirs('output/test')

In [ ]:
!ls ../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian

In [ ]:
img_loc = '../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/'

split_folders.ratio(img_loc, output='output', seed=1, ratio=(0.75, 0.15, 0.1))

In [ ]:
!ls output

In [ ]:
train_loc = 'output/train/'
val_loc = 'output/val/'
test_loc = 'output/test/'

In [ ]:
droprate=0.1
filter_pixel=3
input_shape = (224, 224, 3)
classifier = Sequential()

In [ ]:
#convolution 1st layer
classifier.add(Convolution2D(32, kernel_size=(filter_pixel, filter_pixel), input_shape = input_shape, activation = 'relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#convolution 2nd layer
classifier.add(Convolution2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#convolution 3rd layer
classifier.add(Convolution2D(128, kernel_size=(filter_pixel, filter_pixel), activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#convolution 4th layer
classifier.add(Convolution2D(256, kernel_size=(filter_pixel, filter_pixel), activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#Fully connected 1st layer
classifier.add(Flatten()) 
classifier.add(Dense(4048)) 
#classifier.add(BatchNormalization())
classifier.add(Activation('relu')) 
#classifier.add(Dropout(droprate))

#Fully connected layers
classifier.add(Dense(4048)) 
classifier.add(Activation('relu')) 
classifier.add(Dense(2024)) 
classifier.add(Activation('relu')) 

#Fully connected final layer
classifier.add(Dense(5))
classifier.add(Activation('softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_loc,
                                                 target_size = (224, 224),
                                                 batch_size = 32)

val_set = test_datagen.flow_from_directory(val_loc,
                                            target_size = (224, 224),
                                            batch_size = 32)

In [ ]:
hist=classifier.fit_generator(training_set,
                         steps_per_epoch=training_set.samples//training_set.batch_size, 
                         validation_data=val_set,
                         validation_steps=val_set.samples//val_set.batch_size,
                         epochs=100)


In [ ]:
classifier.save('DR_CNN2.h5') 

In [ ]:
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.title('CNN: Loss and Validation Loss (0.0001 = Adam LR)')
plt.legend();
plt.show()

plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.title('CNN: Accuracy and Validation Accuracy (0.0001 = Adam LR)')
plt.legend();
plt.show()

In [ ]:
test_datagene = ImageDataGenerator(rescale = 1./255)
test_sets = test_datagene.flow_from_directory(test_loc,target_size = (224, 224))
X_test, y_test = test_sets.next()

In [ ]:
score = classifier.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

prediction = classifier.predict(X_test)
test_result = np.argmax(y_test, axis=1)
prediction_result = np.argmax(prediction, axis=1)
confusion__matrix=confusion_matrix(test_result, prediction_result)
print(classification_report(test_result, prediction_result))
print(confusion__matrix)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(confusion__matrix, range(5), range(5))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4)
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16})

plt.show()

In [ ]:
from IPython.display import FileLink


In [ ]:
FileLink(‘DR_CNN2.pth’)

In [ ]:
!ls
